In [4]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import requests
from pandasql import sqldf
import json

In [5]:
print("Which Contest would you like to view?")
contest = input()

Which Contest would you like to view?


In [6]:
page = "https://codeforces.com/api/contest.status?contestId=" + contest + "&from=1"
print(page)

https://codeforces.com/api/contest.status?contestId=1676&from=1


In [7]:
cf_submissions_api = requests.get(page)

In [8]:
submissions = cf_submissions_api.json()

In [9]:
df = pd.json_normalize(submissions,['result'])

In [10]:
df.head(10)

,id,contestId,creationTimeSeconds,relativeTimeSeconds,programmingLanguage,verdict,testset,passedTestCount,timeConsumedMillis,memoryConsumedBytes,...,problem.type,problem.rating,problem.tags,author.contestId,author.members,author.participantType,author.ghost,author.startTimeSeconds,author.teamId,author.teamName
0,157619601,1676,1652927271,171,GNU C++17,OK,TESTS,10,15,1024000,...,PROGRAMMING,800,[implementation],1676,[{'handle': 'Dear_Alokit'}],VIRTUAL,False,1652927100,NaN,NaN
1,157619572,1676,1652927226,2147483647,PyPy 3,OK,TESTS,31,1669,31846400,...,PROGRAMMING,1100,"[binary search, greedy, sortings]",1676,[{'handle': 'Pulga'}],PRACTICE,False,1652193900,NaN,NaN
2,157619555,1676,1652927199,2147483647,GNU C++17,OK,TESTS,12,46,1228800,...,PROGRAMMING,1000,"[brute force, greedy, implementation]",1676,[{'handle': 'ngtvu278'}],PRACTICE,False,1652193900,NaN,NaN
3,157619349,1676,1652926804,2147483647,GNU C++17,OK,TESTS,16,202,15667200,...,PROGRAMMING,1500,"[data structures, divide and conquer, sortings]",1676,[{'handle': 'Tanzin_Ahammad'}],PRACTICE,False,1652193900,NaN,NaN
4,157619298,1676,1652926718,2147483647,GNU C++17,OK,TESTS,13,15,1228800,...,PROGRAMMING,1400,[brute force],1676,[{'handle': 'vishal_nitb'}],PRACTICE,False,1652193900,NaN,NaN
5,157619292,1676,1652926707,2147483647,Python 3,OK,TESTS,6,46,4505600,...,PROGRAMMING,800,"[greedy, math, sortings]",1676,[{'handle': 'xanemi'}],PRACTICE,False,1652193900,NaN,NaN
6,157619241,1676,1652926637,2837,GNU C11,OK,TESTS,31,702,5324800,...,PROGRAMMING,1100,"[binary search, greedy, sortings]",1676,[{'handle': 'bobbilyking'}],VIRTUAL,False,1652923800,NaN,NaN
7,157619211,1676,1652926589,2789,GNU C11,WRONG_ANSWER,TESTS,0,0,5324800,...,PROGRAMMING,1100,"[binary search, greedy, sortings]",1676,[{'handle': 'bobbilyking'}],VIRTUAL,False,1652923800,NaN,NaN
8,157619190,1676,1652926563,2763,GNU C11,WRONG_ANSWER,TESTS,0,15,5324800,...,PROGRAMMING,1100,"[binary search, greedy, sortings]",1676,[{'handle': 'bobbilyking'}],VIRTUAL,False,1652923800,NaN,NaN
9,157619174,1676,1652926536,2736,GNU C11,TIME_LIMIT_EXCEEDED,TESTS,0,3500,5324800,...,PROGRAMMING,1100,"[binary search, greedy, sortings]",1676,[{'handle': 'bobbilyking'}],VIRTUAL,False,1652923800,NaN,NaN


In [11]:
byTimeMemory = df[["relativeTimeSeconds","contestId","problem.index","programmingLanguage","verdict","timeConsumedMillis", "memoryConsumedBytes"]]

In [12]:
#to facilitate the sql queries
byTimeMemory = byTimeMemory.rename(columns = {"timeConsumedMillis": "Time","problem.index":"problem_index"})

This part finds the average runtime of accepted submissions per question per language

In [13]:
runtime = sqldf("select problem_index, programmingLanguage, Time, memoryConsumedBytes from byTimeMemory where verdict = 'OK' ")

In [14]:
runtime.head(10)

,problem_index,programmingLanguage,Time,memoryConsumedBytes
0,A,GNU C++17,15,1024000
1,E,PyPy 3,1669,31846400
2,D,GNU C++17,46,1228800
3,H2,GNU C++17,202,15667200
4,H1,GNU C++17,15,1228800
5,B,Python 3,46,4505600
6,E,GNU C11,702,5324800
7,E,GNU C++20 (64),78,2457600
8,C,GNU C++17,15,1024000
9,B,GNU C++14,62,1024000


In [15]:
runtime["Memory(KB)"] = runtime["memoryConsumedBytes"] // 1024
runtime = runtime.drop(columns = ["memoryConsumedBytes"])
runtime

,problem_index,programmingLanguage,Time,Memory(KB)
0,A,GNU C++17,15,1000
1,E,PyPy 3,1669,31100
2,D,GNU C++17,46,1200
3,H2,GNU C++17,202,15300
4,H1,GNU C++17,15,1200
...,...,...,...,...
122915,A,GNU C++20 (64),15,0
122916,A,GNU C++20 (64),15,1600
122917,A,GNU C++20 (64),15,0
122918,A,GNU C++14,15,0


In [16]:
#example
#runtime.loc[(runtime["problem_index"] == "G") & (runtime["programmingLanguage"] == "PyPy 3-64")]

In [17]:
#runtime_groupby = runtime.groupby(by = ["contestId", "problem_index", "programmingLanguage"]).mean().round(1)

In [18]:
#runtime_groupby

In [20]:
runtime.to_csv("F:/" + contest + "_runtime_stats.csv")